In [1]:
import numpy as np
import torch
import os

## Part 4: Voxelwise Encoding Models

Now that we have our stimuli features from the BridgeTower model, we can use them to predict brain responses. To do this, we will need to load in our fMRI data.
To start we will be using sub1 from Popham et al 2021

### 1 Load fMRI data

In [7]:
s1_movie_train = np.load("data/fmri_data/moviedata/S1/train.npy")

In [4]:
s1_alternateithicatom = np.load("data/fmri_data/storydata/S1/alternateithicatom.npy")
s1_avatar = np.load("data/fmri_data/storydata/S1/avatar.npy")
s1_howtodraw = np.load("data/fmri_data/storydata/S1/howtodraw.npy")
s1_legacy = np.load("data/fmri_data/storydata/S1/legacy.npy")
s1_life = np.load("data/fmri_data/storydata/S1/life.npy")
s1_myfirstdaywiththeyankees = np.load("data/fmri_data/storydata/S1/myfirstdaywiththeyankees.npy")
s1_naked = np.load("data/fmri_data/storydata/S1/naked.npy")
s1_odetostepfather = np.load("data/fmri_data/storydata/S1/odetostepfather.npy")
s1_souls = np.load("data/fmri_data/storydata/S1/souls.npy")

The data is organized as such:

`moviedata`: Inside this folder, there are subfolders for each participant’s data as well as one folder for the stimuli

Within each partcipant’s folder, there is a train.npy and a test.npy file, which contain the brain data for the training and testing data for the models. The shape of each of these is *[n_TR, z, x, y]*, and the last three dimensions should be flattened in order to match the shape of the vertex to voxel mappings in the mappers folder. Data that is located outside of the cortical mask is indicated by np.nan values.
The shape of the data in the stimuli folder is [n_TR, n_features]. The features dimension includes the motion energy features (N=2139), which are then followed by the semantic features (N=985).

`storydata`: Inside this folder, there are subfolders for each participant’s data as well as one folder for the stimuli

Within each participant’s folder, there is a file for each stimulus which contains the brain data. The shape of each of these is *[n_TR, z, x, y]*, and the last three dimensions should be flattened in order to match the shape of the vertex to voxel mappings in the mappers folder. Data that is located outside of the cortical mask is indicated by np.nan values.
The shape of the data in the stimuli folder is [n_TR, n_features]. The features dimension includes the low level features (number of words (N=1), number of phonemes (N=1), and phoneme counts (N=39)), which are then followed by the semantic features (N=985).

In [8]:
mask = ~np.isnan(s1_movie_train)

# Apply the mask and then flatten
# This will keep only the non-NaN values
s1_movie_fmri = s1_movie_train[mask].reshape(s1_movie_train.shape[0], -1)

In [10]:
s1_movie_fmri.shape

(3600, 81111)

In [14]:
s1_movie_fmri[:5, :5]

array([[ 1.2697605 ,  1.8778311 ,  1.896819  , -0.33944297, -0.24112298],
       [ 1.1698035 ,  0.53661877,  0.19726549,  0.47479075, -0.02730744],
       [ 0.8348491 ,  0.49176377, -0.8683965 , -0.53910434,  0.06158145],
       [-1.3390145 , -0.02573461, -0.0707036 ,  0.11303766,  0.0237655 ],
       [ 0.90123653,  0.7940609 ,  1.312021  ,  0.47119457, -0.03368266]],
      dtype=float32)

In [15]:
def voxelwise_normalization(fmri_data):
    # Calculate mean and standard deviation for each voxel
    mean_per_voxel = np.mean(fmri_data, axis=0)
    std_per_voxel = np.std(fmri_data, axis=0)

    # Perform voxel-wise normalization
    normalized_fmri_data = (fmri_data - mean_per_voxel) / std_per_voxel

    return normalized_fmri_data

In [16]:
normal_fmri = voxelwise_normalization(s1_movie_fmri)

In [17]:
normal_fmri[:5,:5]

array([[ 1.3318814 ,  1.9632717 ,  1.9818691 , -0.37418306, -0.27609047],
       [ 1.2262138 ,  0.5536382 ,  0.19786882,  0.47834775, -0.04839148],
       [ 0.8721232 ,  0.50649494, -0.9207434 , -0.5832354 ,  0.04626913],
       [-1.4259351 , -0.03740335, -0.08341502,  0.0995798 ,  0.00599771],
       [ 0.94230336,  0.82421356,  1.3680139 ,  0.47458243, -0.05518066]],
      dtype=float32)

### 2 Load in feature vectors

In [13]:
# movie data
train00 = np.load("results/feature_vectors/movie/train_00_data.npy")
train01 = np.load("results/feature_vectors/movie/train_01_data.npy")
train02 = np.load("results/feature_vectors/movie/train_02_data.npy")
train03 = np.load("results/feature_vectors/movie/train_03_data.npy")
train04 = np.load("results/feature_vectors/movie/train_04_data.npy")
train05 = np.load("results/feature_vectors/movie/train_05_data.npy")
train06 = np.load("results/feature_vectors/movie/train_06_data.npy")
train07 = np.load("results/feature_vectors/movie/train_07_data.npy")
train08 = np.load("results/feature_vectors/movie/train_08_data.npy")
train09 = np.load("results/feature_vectors/movie/train_09_data.npy")
train10 = np.load("results/feature_vectors/movie/train_10_data.npy")
train11 = np.load("results/feature_vectors/movie/train_11_data.npy")

In [14]:
# We need to combine all the training vectors so that they correspond to the fmri training dataset
movie_features = np.vstack((train00, train01, train02, train03, train04, train05, train06, train07, train08, train09, train10, train11))
movie_features.shape

(3870, 768)

In [17]:
# story data
alternateithicatom = np.load("results/feature_vectors/story/alternateithicatom_data.npy")
avatar = np.load("results/feature_vectors/story/avatar_data.npy")
howtodraw = np.load("results/feature_vectors/story/howtodraw_data.npy")
legacy = np.load("results/feature_vectors/story/legacy_data.npy")
life = np.load("results/feature_vectors/story/life_data.npy")
myfirstdaywiththeyankees = np.load("results/feature_vectors/story/myfirstdaywiththeyankees_data.npy")
naked = np.load("results/feature_vectors/story/naked_data.npy")
odetostepfather = np.load("results/feature_vectors/story/odetostepfather_data.npy")
souls = np.load("results/feature_vectors/story/souls_data.npy")

### 3 Voxelwise Encoding Model for Movie Data
We want to create a voxelwise encoding model that captures the relationship between stimuli features from bridgetower and the fmri responses. 

Since our movie data was create with the same temporal alignment (TR = 2) as our fMRI data, we do not need to do any resampling.

#### 3.1 Finite impulse response model
Understanding the Delays: fMRI responses to stimuli are not instantaneous but follow the hemodynamic response function (HRF), which peaks around 4-6 seconds after the stimulus. By selecting delays of 2, 4, 6, and 8 seconds, you're aiming to capture the rise, peak, and fall of this hemodynamic response relative to your features.

In [18]:
# Assuming features is your (3600, 768) feature data
# And TR (time between scans) is 2 seconds
delays = [2, 4, 6, 8]  # Delays in seconds
shifted_features_list = []

for delay in delays:
    shift_amount = delay // 2  # Assuming TR is 2 seconds
    shifted = np.roll(movie_features, shift_amount, axis=0)
    # Optionally, handle edge effects here (e.g., zero-padding or trimming)
    shifted_features_list.append(shifted)

# Stack the shifted arrays to create a 3D array
shifted_features_3d = np.stack(shifted_features_list, axis=-1)

#### 3.2 Linear model

In [19]:
# We will be using L2-regularized linear regression
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import KFold

For each of the 81111 vertices, we want to calculate it's relationship with the 4*768 delayed features. Where k=768 and m is our vertex of interest:

`m = 4k * beta`

So, after running regression, we should have a matrix of m # of rows and 4k # of columns, where each cell contains the beta values or linear weights on the 4k delayed features for each voxel.

We are told in the dataset that data that is located outside of the cortical mask is indicated by np.nan values so we should ignore these in our regression

In [20]:
# Assuming 'features_shifted' is your 3D array (time points x features x delays)
# And 'fmri_data' is your 2D fMRI data (time points x voxels)

# Reshape the feature data for regression
n_time_points, n_features, n_delays = shifted_features_3d.shape
features_reshaped = shifted_features_3d.reshape(n_time_points, n_features * n_delays)

In [21]:
features_reshaped.shape

(3870, 3072)

In [22]:
n_voxels = s1_movie_fmri.shape[1]
alphas = np.logspace(-6, 6, 50) # Range for alphas
kf = KFold(n_splits=5) # Example of 5-fold cross-validation

best_alphas = np.zeros(n_voxels)
coefficients = np.zeros((n_voxels, features_reshaped.shape[1]))

In [23]:
import warnings
warnings.filterwarnings("ignore")

for voxel in range(n_voxels):
    ridge_cv = RidgeCV(alphas=alphas, cv=kf)
    ridge_cv.fit(features_reshaped, s1_movie_fmri[:, voxel])
    best_alphas[voxel] = ridge_cv.alpha_
    coefficients[voxel, :] = ridge_cv.coef_

    #alphas = np.array(best_alphas)
    #coef = np.array(coefficients)
    np.save('results/encoding_model/movie/best_alphas.npy', best_alphas)
    np.save('results/encoding_model/movie/coefficients.npy', coefficients)
    # Print checkpoints
    if voxel == round(n_voxels*0.1):
        print("10% done")
    elif voxel == round(n_voxels*0.2):
        print("20% done")
    elif voxel == round(n_voxels*0.3):
        print("30% done")
    elif voxel == round(n_voxels*0.4):
        print("40% done")
    elif voxel == round(n_voxels*0.5):
        print("50% done")
    elif voxel == round(n_voxels*0.6):
        print("60% done")
    elif voxel == round(n_voxels*0.7):
        print("70% done")
    elif voxel == round(n_voxels*0.8):
        print("80% done")
    elif voxel == round(n_voxels*0.9):
        print("90% done")
print("Complete")

#### 3.3 Running voxelwise encoding models on HPC cluster
Due to the computational demands of running 81,111 voxels through 50 iterations of cross-validated ridge regression, we ran this on an HPC cluster in batches. Code for the containerized image of python and the voxelwise encoding script can be found in `container`. Results are still found in `results/encoding_model` but are listed in batches: coefficients_0-1599.npy corresponds to the coefficients for the first 1600 voxels.